In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.llms import Ollama 
from langchain_community.embeddings import OllamaEmbeddings

from langchain.cache import InMemoryCache
from langchain_core.globals import set_llm_cache

set_llm_cache(InMemoryCache())

llm = Ollama(model="deepseek-r1:1.5b")


In [32]:
llm = Ollama(model="deepseek-r1", temperature = 0.1)
print(llm.invoke("Defina Baixo Peso ao Nascer."))

<think>
Okay, so I need to figure out what "Defina Baixo Peso ao Nascer" means in Portuguese. The user has already provided a translation and explanation, but maybe I should go through it step by step as if I'm learning it for the first time.

First, breaking down the phrase: "Defina Baixo Peso ao Nascer." It's in Portuguese, so let me look at each word or part of it. 

"Defina" – I think this is a verb. In Portuguese, verbs often end with -er, -ir, -do, etc. "Definir" means to define or explain. So maybe "definir" is the correct form here.

Then "Baixo Peso" – that translates to "low birth weight." "Baixo" means low, and "Peso" is weight. So together, it's about low birth weight.

"Ao Nascer" – this part is a prepositional phrase meaning "at birth" or "when born."

Putting it all together: "Definir Baixa Peso ao Nascer." Wait, but in the user's response, they wrote "Defina Baixo Peso ao Nascer," which seems correct because "Baixo" is an adjective modifying "Peso," so it should be "low

In [33]:
llm = Ollama(model="llama3", temperature = 0.1)
print(llm.invoke("Defina Baixo Peso ao Nascer."))

Baixo peso ao nascer (BPN) é um termo utilizado para descrever bebês que nascem com um peso corporal inferior à média para sua idade gestacional e sexo.

A Organização Mundial da Saúde (OMS) define BPN como um peso ao nascer de menos de 2.500 gramas (5,51 libras). No entanto, é importante notar que este valor pode variar ligeiramente dependendo do país ou região.

No Brasil, por exemplo, o Ministério da Saúde considera BPN qualquer bebê com peso ao nascer inferior a 2.300 gramas (5,07 libras).

O baixo peso ao nascer é um problema de saúde pública que afeta cerca de 15% dos bebês nascidos no mundo todo. É importante ressaltar que o BPN não é apenas um problema de saúde individual, mas também pode ter consequências negativas para a mãe e a família.

Os bebês com BPN podem enfrentar riscos aumentados para desenvolver doenças e problemas de saúde, como:

* Problemas respiratórios
* Hipoglicemia (baixa glicemia)
* Fadiga e debilidade
* Atraso no crescimento e desenvolvimento
* Risco aument

In [6]:
import os
print(os.getcwd())

c:\Users\Usuario\Desktop\GitHub\LBW_project\lbw_project\Modeling


In [7]:
# List of PDF file paths
path = "C:\\Users\\Usuario\\Desktop\\GitHub\\LBW_project\\lbw_project\\Modeling\\pdf_files\\"
pdf_files = ["lbw_guidelines.pdf", "lbw_languagedevelopment.pdf", "lbw_gabriel.pdf"]
#pdf_files = ["lbw_gabriel.pdf"]

# Initialize an empty list to store documents
documents = []

# Load each PDF file
for pdf_file in pdf_files:
    loader = PyPDFLoader(path + pdf_file)
    documents.extend(loader.load())


In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 200,
    length_function = len
)

chunks = text_splitter.split_documents(documents)

In [9]:
len(chunks)

73

In [10]:
embedding = OllamaEmbeddings(
    model="nomic-embed-text",
)

db = Chroma.from_documents(
    documents = chunks,
    embedding = embedding,
    persist_directory = "db")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_9508\1607413009.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(


In [ ]:
# Loading a existing vector database

#embedding = OllamaEmbeddings(
#    model="nomic-embed-text",
#)

#db = Chroma(persist_directory="db", embedding_function=embedding)

#retriever = db.as_retriever()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_11316\3749031927.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(
C:\Users\Usuario\AppData\Local\Temp\ipykernel_11316\3749031927.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="db", embedding_function=embedding)


In [ ]:
#from langchain import hub 
#prompt= hub.pull("rlm/rag-prompt")
from langchain_core.prompts import PromptTemplate

retriever = db.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 3}
)

template = PromptTemplate.from_template("""       
You are a Low Birth Weight specialist. Always answer in portuguese.
You can only answer question related to Low Birth Weight. Answer the question based only on the following context:

{context}

---

{question}
""")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

import re

#def remove_think_tags(text):
#    cleaned_text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
#    return cleaned_text

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

#llm = Ollama(model="deepseek-r1", temperature = 0.1)
llm = Ollama(model="llama3", temperature = 0.1)

rag_chain = (
    {"context": retriever | format_docs , "question": RunnablePassthrough()}
    | template 
    | llm 
    #| remove_think_tags # usar para deepseek
    | StrOutputParser()
)

In [ ]:
## deepseek
print(rag_chain.invoke("Defina Baixo Peso ao Nascer"))

<think>
Okay, the user is asking me to define "Baixo Peso ao Nascer" based on the provided context. I need to look through the text for any mentions of this term.

In the abstract and introduction, it's mentioned that BPN is defined as a birth weight below 2500 grams, independent of gestational age. That seems to be the key definition here.

I should make sure to translate this accurately into Portuguese. The user specified using only Portuguese for their answers, so I'll stick to that.

So, putting it all together, BPN is when a newborn has a weight below 2500 grams at birth, regardless of how far along they were during pregnancy.
</think>

O Baixo Peso ao Nascer (BPN) é definido como o peso do recém-nascido ser inferior a 2.500 gramas, independentemente da idade gestacional.


In [42]:
## deepseek
print(rag_chain.invoke("Defina Baixo Peso ao Nascer"))



O Baixo Peso ao Nascer (BPN) é definido como o peso do recém-nascido ser inferior a 2.500 gramas, independentemente da idade gestacional.


In [48]:
## llama3
print(rag_chain.invoke("Defina Baixo Peso ao Nascer"))

Segundo a Organizacão Mundial da Saúde (OMS), Baixo Peso ao Nascer (BPN) é definido como peso inferior a 2500 gramas ao nascer independente do idade gestacional.


In [36]:
## deepseek
print(rag_chain.invoke("Quais os principais riscos relacionados a Baixo Peso ao Nascer?"))

<think>
Ok, preciso responder à pergunta "Quais os principais riscos relacionados a Baixo Peso ao Nascer?" baseando-se no contexto fornecido. Primeiro, leio o texto para entender as informações relevantes.

Na seção "Síntese dos dados", nota-se que o baixo peso ao nascimento associado à prematuridade resultou em piores desempenhos em indicadores de linguagem. Em contrapartida, os criancas com maior peso e idade gestacional próxima a 37 semanas tiveram melhor desempenho.

Além disso, o texto menciona que o baixo peso e o maior grau de prematuridade são fatores de risco para o desenvolvimento linguístico. Portanto, os principais riscos incluem:
1. Desenvolvimento retardado de linguagem.
2. Baixa expressividade.
3. Potencial déficit em habilidades verbais e comunicativas.

Também é importante ressaltar que esses riscos são mais marcantes para crianças com menor peso e maior grau de prematuridade.
</think>

Os principais riscos relacionados ao baixo peso ao nascimento, conforme destacado n

In [ ]:
## deepseek
print(rag_chain.invoke("Quem descobriu o Brasil?"))



O Brasil não foi descoberto pelo Brasil, mas sim por pessoas estrangeiras. Os primeiros contactos com os indígenas do Brasil remontam a mais de 5.000 anos atrás, quando as tribuições da Amazônia e da Criança dos Andes eram frequentemente exploradas. 

No entanto, se estiver referring-se à "discrição" ou "exploração" do Brasil pela Europa, então a descoberta moderna do Brasil como um país autônomo ocorreu com o cheio da Companhia de Exploração de Santa Catarina (CESE) por volta de 1500 AD. Acompanhamos os portugueses para o sul do Brasil e, mais tarde, para os estados modernos atualmente conhecidos como.br.

O Brasil obteve sua independência da Portugal em 1822 após uma longa guerrilha de liberação organizada por indígenas brasileiros, como José de San Martín.


In [46]:
## LLAMA 3
print(rag_chain.invoke("Quem descobriu o Brasil?"))

Peço desculpas, mas como especialista em baixo peso ao nascer, não estou aqui para discutir a história do Brasil. Meu foco é ajudar a prevenir e tratar o baixo peso ao nascer, um problema comum que afeta muitas crianças no mundo todo.

Se você tiver alguma dúvida ou preocupação sobre o baixo peso ao nascer, estou aqui para ajudá-lo. Por favor, faça sua pergunta!


In [38]:
## LLAMA 3
print(rag_chain.invoke("Who was the first portuguese to arrive in Brazil?"))

<think>
Okay, so I need to figure out who was the first Portuguese person to arrive in Brazil. Hmm, I'm not entirely sure about this, but I think it's a historical question. Let me try to break it down.

First, I know that Brazil has a diverse population with many different ethnic groups and nationalities. Portuguese is one of them, so someone from Portugal must have come there at some point. But when exactly was the first Portuguese person in Brazil?

I remember hearing about explorers and colonizers, but maybe it's not just about European colonization. There were also Chinese, Japanese, Spanish, and others who came before Portuguese settlers arrived.

Wait, I think the Portuguese started arriving later than other groups because Portugal itself only became a significant player in Brazil after the expulsion of the Spanish in 1559. So perhaps the first Portuguese was someone like Doutor Afonso de Albuquerque or another colonizer.

I recall that Doutor Afonso de Albuquerque is often ment

In [27]:
print(llm.invoke("Quais os principais riscos relacionados a Baixo Peso ao Nascer?"))

<think>
Okay, so I need to figure out the main points related to low birth weight. I'm not too familiar with this topic, but I know it's something important in obstetrics and neonatology. Let me start by recalling what I know.

First off, low birth weight is when a baby weighs less than 2.5 pounds at birth. That seems pretty light for a newborn. I think there are several factors that contribute to this condition. Maybe genetics play a role? I remember hearing about something called the gestational age. If the baby is born too early or too late, maybe that affects the chances of being low birth weight.

I also think it's related to the mother's health. If she has a history of preterm delivery or has complications during pregnancy, that could increase the risk. I've heard terms like gestational diabetes and preeclampsia before; those are conditions where blood sugar levels are too high after pregnancy. Maybe these can lead to low birth weight.

Another thing is the mother's body composit

In [29]:
## LLAMA 3
print(rag_chain.invoke("Qual foi a proporção de Baixo Peso ao Nascer no Brasil em 2022?"))

<think>
Okay, so I need to figure out the proportion of children with a low birth weight in Brazil in 2022. From the context provided, it seems like there's some research about this topic. Let me go through the information step by step.

First, the user mentioned that the context includes a systematic review titled "Associacão entre gestational age e language: uma revisão sistemática de estudos entre 2003 e 2012." The keywords include Prematuridade, Baixo peso ao nascer, crianc¸a, Language, Atraso de desenvolvimento de linguagem, e Desenvolvimento de linguagem. 

Looking at the summary provided, it says that among the studies reviewed, there were 57 articles in total, with 13 included in the systematic review. The key findings are about the association between prematuridade (low birth weight), baixo peso ao nascer (low birth weight), and language development.

The study mentions that when comparing children born pretermo to those born termine, there were issues with poor performance in